# Train a JointVAE model

In [1]:
%reload_ext autoreload
%autoreload 2
%matplotlib inline

In [2]:
import torch
from viz.visualize import Visualizer
use_cuda = torch.cuda.is_available()
use_cuda

True

In [3]:
import os
os.environ["CUDA_DEVICE_ORDER"] = "PCI_BUS_ID"  # see issue #152
os.environ["CUDA_VISIBLE_DEVICES"] = "0"  # change to your device

#### Prepare data list

In [4]:
!ls data

dress_dresslen_train_test_splits.json	dress_sleeve_train_test_splits.json
dress_sleevelen_train_test_splits.json


In [5]:
import json
with open("./data/dress_dresslen_train_test_splits.json", "r") as infile:
    data_dict = json.load(infile)

In [6]:
data_dict.keys()

dict_keys(['X_train_1', 'y_train_1', 'X_test_1', 'y_test_1', 'X_train_2', 'y_train_2', 'X_test_2', 'y_test_2', 'X_train_3', 'y_train_3', 'X_test_3', 'y_test_3', 'X_train_4', 'y_train_4', 'X_test_4', 'y_test_4', 'X_train_5', 'y_train_5', 'X_test_5', 'y_test_5', 'X_train_6', 'y_train_6', 'X_test_6', 'y_test_6', 'X_train_7', 'y_train_7', 'X_test_7', 'y_test_7', 'X_train_8', 'y_train_8', 'X_test_8', 'y_test_8', 'X_train_9', 'y_train_9', 'X_test_9', 'y_test_9', 'X_train_10', 'y_train_10', 'X_test_10', 'y_test_10'])

In [7]:
data_dict['X_train_1'][:5]

['/2/8/2893552_3773662.jpg',
 '/2/9/2982376_3889235.jpg',
 '/2/7/2783355_3578973.jpg',
 '/2/9/2974380_3918638.jpg',
 '/2/8/2886740_3675612.jpg']

#### Create list of image paths

In [8]:
!python -V

Python 3.6.5


In [9]:
image_paths_train = []
image_paths_test = []

root_data_dir = "/home/jovyan/vmldata/raw_source_data/v20180810_all_wearables"

for key, val in data_dict.items():
    if 'X_train' in key:
        image_paths_train.extend([root_data_dir + imgpath for imgpath in val])
    elif 'X_test' in key:
        image_paths_test.extend([root_data_dir + imgpath for imgpath in val])

print(f"Number of train image paths: {len(image_paths_train):,d}")
print(f"Number of test image paths: {len(image_paths_test):,d}")
print()
print("Sample paths:")
print(image_paths_train[0])
print(image_paths_train[-1])
print(image_paths_test[0])
print(image_paths_test[-1])

Number of train image paths: 167,742
Number of test image paths: 18,638

Sample paths:
/home/jovyan/vmldata/raw_source_data/v20180810_all_wearables/2/8/2893552_3773662.jpg
/home/jovyan/vmldata/raw_source_data/v20180810_all_wearables/2/6/2683298_3676896.jpg
/home/jovyan/vmldata/raw_source_data/v20180810_all_wearables/2/4/2431229_3158108.jpg
/home/jovyan/vmldata/raw_source_data/v20180810_all_wearables/2/5/2569101_3223742.jpg


In [10]:
#from utils.dataloaders import get_mnist_dataloaders, get_fashion_mnist_dataloaders
#train_loader, test_loader = get_mnist_dataloaders(batch_size=64)
#train_loader, test_loader = get_fashion_mnist_dataloaders(batch_size=64)

In [10]:
from torchvision import transforms
from utils.dataloaders_custom import get_imagelist_dataloader, ImageListDataset

BATCH_SIZE = 128

#composed = transforms.Compose([transforms.Resize((260,260)), transforms.ToTensor()])
composed = transforms.Compose([transforms.Resize((64,64)), transforms.ToTensor()])

train_dataset = ImageListDataset(image_paths_train, transform=composed)
test_dataset = ImageListDataset(image_paths_test, transform=composed)

train_loader = get_imagelist_dataloader(batch_size=BATCH_SIZE, dataset_object=train_dataset)
test_loader = get_imagelist_dataloader(batch_size=BATCH_SIZE, dataset_object=test_dataset)

### Define latent distribution of the model

In [11]:
# Latent distribution will be joint distribution of 10 gaussian normal distributions
# and one 10 dimensional Gumbel Softmax distribution
latent_spec = {'cont': 10,
               'disc': [10]}

### Build a model

In [12]:
#from jointvae.models_v1 import VAE
from jointvae.models import VAE

#model = VAE(latent_spec=latent_spec, img_size=(3, 260, 260), use_cuda=use_cuda)
model = VAE(latent_spec=latent_spec, img_size=(3, 64, 64), use_cuda=use_cuda)

In [13]:
print(model)

VAE(
  (img_to_features): Sequential(
    (0): Conv2d(3, 32, kernel_size=(4, 4), stride=(2, 2), padding=(1, 1))
    (1): ReLU()
    (2): Conv2d(32, 32, kernel_size=(4, 4), stride=(2, 2), padding=(1, 1))
    (3): ReLU()
    (4): Conv2d(32, 64, kernel_size=(4, 4), stride=(2, 2), padding=(1, 1))
    (5): ReLU()
    (6): Conv2d(64, 64, kernel_size=(4, 4), stride=(2, 2), padding=(1, 1))
    (7): ReLU()
  )
  (features_to_hidden): Sequential(
    (0): Linear(in_features=1024, out_features=256, bias=True)
    (1): ReLU()
  )
  (fc_mean): Linear(in_features=256, out_features=10, bias=True)
  (fc_log_var): Linear(in_features=256, out_features=10, bias=True)
  (fc_alphas): ModuleList(
    (0): Linear(in_features=256, out_features=10, bias=True)
  )
  (latent_to_features): Sequential(
    (0): Linear(in_features=20, out_features=256, bias=True)
    (1): ReLU()
    (2): Linear(in_features=256, out_features=1024, bias=True)
    (3): ReLU()
  )
  (features_to_img): Sequential(
    (0): ConvTranspose

### Train the model

In [14]:
from torch import optim

# Build optimizer
optimizer = optim.Adam(model.parameters(), lr=3e-4, amsgrad=True) # added amsgrad # orig lr 5e-4

In [15]:
from jointvae.training import Trainer

# Define the capacities
# Continuous channels
cont_capacity = [0.0, 5.0, 25000, 30.0]  # Starting at a capacity of 0.0, increase this to 5.0
                                         # over 25000 iterations with a gamma of 30.0
# Discrete channels
disc_capacity = [0.0, 5.0, 25000, 30.0]  # Starting at a capacity of 0.0, increase this to 5.0
                                         # over 25000 iterations with a gamma of 30.0

# Build a trainer
trainer = Trainer(model, optimizer,
                  cont_capacity=cont_capacity,
                  disc_capacity=disc_capacity,
                 use_cuda=use_cuda)

#### Initialize visualizer

In [16]:
# Build a visualizer which will be passed to trainer to visualize progress during training
viz = Visualizer(model)

In [ ]:
# Train model for 10 epochs
# Note this should really be a 100 epochs and trained on a GPU, but this is just to demo

trainer.train(train_loader, epochs=200, save_training_gif=('./training_rd64x64_e200_b128.gif', viz))

0/167742	Loss: 2831.572
6400/167742	Loss: 2417.275
12800/167742	Loss: 1492.403
19200/167742	Loss: 1394.205
25600/167742	Loss: 1362.082
32000/167742	Loss: 1347.232
38400/167742	Loss: 1333.981
44800/167742	Loss: 1333.034
51200/167742	Loss: 1323.519
57600/167742	Loss: 1320.870
64000/167742	Loss: 1305.000
70400/167742	Loss: 1305.948
76800/167742	Loss: 1290.296
83200/167742	Loss: 1293.515
89600/167742	Loss: 1277.715
96000/167742	Loss: 1274.329
102400/167742	Loss: 1266.086
147200/167742	Loss: 1235.774
153600/167742	Loss: 1232.759
160000/167742	Loss: 1237.104
166400/167742	Loss: 1227.729
Epoch: 1 Average loss: 1337.91
0/167742	Loss: 1225.909
6400/167742	Loss: 1226.462
12800/167742	Loss: 1229.457
19200/167742	Loss: 1229.924
25600/167742	Loss: 1223.729
32000/167742	Loss: 1222.461
38400/167742	Loss: 1222.641
44800/167742	Loss: 1217.927
51200/167742	Loss: 1220.775
57600/167742	Loss: 1214.772
64000/167742	Loss: 1222.621
70400/167742	Loss: 1211.618
76800/167742	Loss: 1223.923
83200/167742	Loss: 121

102400/167742	Loss: 1135.066
108800/167742	Loss: 1128.470
115200/167742	Loss: 1129.962
121600/167742	Loss: 1131.605
128000/167742	Loss: 1127.597
134400/167742	Loss: 1129.201
140800/167742	Loss: 1131.965
147200/167742	Loss: 1134.628
153600/167742	Loss: 1136.473
160000/167742	Loss: 1130.931
166400/167742	Loss: 1129.364
Epoch: 11 Average loss: 1133.53
0/167742	Loss: 1114.932
6400/167742	Loss: 1130.742
12800/167742	Loss: 1125.854
19200/167742	Loss: 1127.512
25600/167742	Loss: 1133.775
32000/167742	Loss: 1130.888
38400/167742	Loss: 1134.755
44800/167742	Loss: 1125.815
51200/167742	Loss: 1131.315
57600/167742	Loss: 1131.392
64000/167742	Loss: 1136.174
70400/167742	Loss: 1129.225
76800/167742	Loss: 1128.345
83200/167742	Loss: 1129.105
89600/167742	Loss: 1133.391
96000/167742	Loss: 1131.670
102400/167742	Loss: 1131.869
108800/167742	Loss: 1131.745
115200/167742	Loss: 1127.347
121600/167742	Loss: 1131.051
128000/167742	Loss: 1129.097
134400/167742	Loss: 1127.836
140800/167742	Loss: 1129.432
147

160000/167742	Loss: 1108.593
166400/167742	Loss: 1107.768
Epoch: 21 Average loss: 1108.64
0/167742	Loss: 1107.178
6400/167742	Loss: 1112.482
12800/167742	Loss: 1104.009
19200/167742	Loss: 1113.057
25600/167742	Loss: 1111.112
32000/167742	Loss: 1106.684
38400/167742	Loss: 1105.450
44800/167742	Loss: 1105.249
51200/167742	Loss: 1105.511
57600/167742	Loss: 1103.406
64000/167742	Loss: 1111.606
70400/167742	Loss: 1107.103
76800/167742	Loss: 1098.819
83200/167742	Loss: 1107.153
89600/167742	Loss: 1111.015
96000/167742	Loss: 1101.297
102400/167742	Loss: 1107.291
108800/167742	Loss: 1110.989
115200/167742	Loss: 1111.016
121600/167742	Loss: 1101.204
128000/167742	Loss: 1116.688
134400/167742	Loss: 1107.441
140800/167742	Loss: 1104.724
147200/167742	Loss: 1111.733
153600/167742	Loss: 1109.464
160000/167742	Loss: 1108.320
166400/167742	Loss: 1110.740
Epoch: 22 Average loss: 1108.26
0/167742	Loss: 1114.038
6400/167742	Loss: 1110.504
12800/167742	Loss: 1103.178
19200/167742	Loss: 1105.872
25600/167

### Visualize

In [ ]:
# Plot reconstructions
%matplotlib inline
import matplotlib.pyplot as plt
import matplotlib.image as mpimg

# Get a batch of data
for batch, labels in test_loader:
    break

# Reconstruct data using Joint-VAE model
recon = viz.reconstructions(batch)

plt.figure(figsize=(20,20))
#np.transpose(recon.numpy(), (2,1,0))
plt.imshow(np.rot90(np.transpose(recon.numpy(),(2,1,0)),k=3));
plt.savefig("sample_images/306/64/e200_b128_reconstructions.png",dpi=600)

In [ ]:
# Plot samples
samples = viz.samples()

plt.figure(figsize=(16,16))
#plt.imshow(samples.numpy()[0, :, :],cmap='gray');
plt.imshow(np.rot90(np.transpose(samples.numpy(),(2,1,0)),k=3));
plt.savefig("sample_images/306/64/e10_b20_samples.png",dpi=600)

In [ ]:
# Plot all traversals
traversals = viz.all_latent_traversals(size=10)

plt.figure(figsize=(16,16))
plt.imshow(np.rot90(np.transpose(traversals.numpy(),(2,1,0)),k=3));
plt.savefig("sample_images/306/64/e10_b20_all_traversals.png",dpi=600)

In [ ]:
# Plot a grid of some traversals
traversals = viz.latent_traversal_grid(cont_idx=2, cont_axis=1, disc_idx=0, disc_axis=0, size=(10, 10))

plt.figure(figsize=(20,20))
plt.imshow(np.rot90(np.transpose(traversals.numpy(),(2,1,0)),k=3));
plt.savefig("sample_images/306/64/e10_b20_traversals2100.png",dpi=600)

In [ ]:
# Plot a grid of some traversals
traversals = viz.latent_traversal_grid(cont_idx=1, cont_axis=1, disc_idx=0, disc_axis=0, size=(10, 10))

plt.figure(figsize=(16,16))
plt.imshow(traversals.numpy()[0, :, :]);
plt.imshow(np.rot90(np.transpose(traversals.numpy(),(2,1,0)),k=3));
plt.savefig("sample_images/306/64/e10_b20_traversals1100.png",dpi=600)

In [ ]:
# Plot a grid of some traversals
traversals = viz.latent_traversal_grid(cont_idx=9, cont_axis=1, disc_idx=0, disc_axis=0, size=(10, 10))

plt.figure(figsize=(16,16))
#plt.imshow(traversals.numpy()[0, :, :], cmap='gray');
plt.imshow(np.rot90(np.transpose(traversals.numpy(),(2,1,0)),k=3));
plt.savefig("sample_images/306/64/e10_b20_traversals9100.png",dpi=600)

### Save Model

In [ ]:
model_name = "realdata64x64_e200_b128.pth"

In [ ]:
torch.save(model.state_dict(),"statedict_" + model_name) # save state dict
torch.save(model, model_name) # save full model

In [ ]:
print("Done training: ",model_name)

#### Restore Model from State Dict

In [28]:
sd_model = VAE(latent_spec=latent_spec, img_size=(3, 64, 64))
sd_model.load_state_dict(torch.load("statedict_" + model_name))

#### Restore Full Model
* Note in this case the serialized data is bound to the specific classes and exact directory strucutre used.

In [ ]:
full_model = torch.load(model_name)

In [ ]:
type(full_model)

In [ ]:
type(sd_model)

In [36]:
!ls

data				requirements.txt
dataloading_pytorch_test.ipynb	statedict_jvae_fmnist_oct292018.pth
imgs				trained_models
jointvae			training.gif
jvae_fmnist_oct292018.pth	training_rd1.gif
latent_traversals.py		train_model.ipynb
load_model.ipynb		train_model_realdata_306_260_v1.ipynb
main.py				train_model_realdata_64_v1.ipynb
__pycache__			utils
RandomUtilsandTests.ipynb	viz
README.md
